In [1]:
import sys
path = '/home/yihuang/Documents/CODE/fractalNet_/cynet2to3/'
sys.path.append(path)
import cynet.cynet as cn
import pandas as pd
import numpy as np
import os
from glob import glob

## Process Event Log File and Generate Split and Triplet

### 1. Parameter setting

#### a. File
In the following cell, we specify the event log input file as `LOGFILE`. Since the `STOREFILE` is only used internally, we can just name it after the `LOGFILE`. 

In [2]:
LOGFILE = f'{path}/data/terror.csv'
STOREFILE = os.path.join(f'{path}/ntb', os.path.basename(LOGFILE).split('.')[0] + '.p')
# In this case, STOREFILE = '/project2/ishanu/YI_terror/ntb/terror.p'

#### b. Spatial Range and Discretization

In the following cell, we specify the tiles used for spatial discretization.
We cut latitude (longitude) between `lat_min` and `lat_max` (`lon_min` and `lon_max`) into `lat_eps` (`lon_eps`) equal parts. Each tiles is one longitude step size wide and one latitude step size high.

In [2]:
# Column names in the event log file for coordinate 1 and 2
coord1, coord2 ='latitude', 'longitude'

# Tiles
lat_min, lat_max = -4, 49
lon_min, lon_max = -16, 84
lat_eps, lon_eps = 10, 10

lat = np.around(np.linspace(lat_min, lat_max, lat_eps + 1), decimals=5)
lon = np.around(np.linspace(lon_min, lon_max, lon_eps + 1), decimals=5)
tiles = [[lat[i], lat[i + 1], lon[j], lon[j + 1]] for i in np.arange(lat_eps) for j in np.arange(lon_eps)]

#### c. Time Range and Discretization

In [3]:
# Column names in the event log file for year, month, and day
year, month, day='iyear', 'imonth', 'iday'
init_date, end_date, freq = '2012-01-01', '2016-12-31', 'D'

#### d. Event
If a time series has an event frequency less than `threshold`, discard the time series.

In [4]:
event_dict = {
    'number_of_kills': {
        'col_name': 'nkill',
        'value_limits': [0, 10000],
        'threshold': 0.025,
        'csvname_prefix': 'NKILL'
    },
    'BEFIA': {
        'col_name': 'attacktype1_txt',
        'types': [[
            'Bombing/Explosion', 
            'Facility/Infrastructure Attack'
        ]],
        'threshold': 0.025,
        'csvname_prefix': 'BEFIA'
    },
    'AAHHH': {
        'col_name': 'attacktype1_txt',
        'types': [[
            'Armed Assault', 
            'Assassination',
            'Hijacking',
            'Hostage Taking (Barricade Incident)',
            'Hostage Taking (Kidnapping)'
        ]],
        'threshold': 0.025,
        'csvname_prefix': 'AAHHH'
    }
}

### 2. Generating Time Series for Training and Test

#### a. Time Series for the Number of kills
Our first fit is `S0` for time series of number of kills. 
Essentially, we are looking for tiles that meet a certain number of kills (deaths in the column `nkill`). 
We are looking for tiles with number of kills that are greater than a certain `threshold`. 
Here that `threshold` is $0.025$.
A file named `NKILL.csv` is outputted. 
And, more importantly, the internal timeseries dataframe is changed.

In [9]:
S0 = cn.spatioTemporal(
    # File
    log_file=LOGFILE,
    log_store=STOREFILE,
    # Spatial
    coord1=coord1,
    coord2=coord2,
    grid=tiles,
    # Temporal
    year=year,
    month=month,
    day=day,
    init_date=init_date,
    end_date=end_date,
    freq=freq,
    # Event
    EVENT=event_dict['number_of_kills']['col_name'],
    value_limits=event_dict['number_of_kills']['value_limits'],
    threshold=event_dict['number_of_kills']['threshold'])

S0.fit(csvPREF=event_dict['number_of_kills']['csvname_prefix'])

/home/yihuang/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (4,6,31,33,53,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
100%|██████████| 100/100 [01:37<00:00,  1.03it/s]


**Note** that we are now going to use the tiles selected for in `S0`. 

In [10]:
tiles = S0.getGrid()

#### b. Time Series for Bombing/Explosion and Facility/Infrastructure Attack
`S1` will be our fitting for attack types in the categories 
 - `Bombing/Explosion` and 
 - `Facility/Infrastructure Attack`.

We are counting the number of these types of events that happen in these tiles.
Output is written to `BEFIA.csv`, which contains the timeseries for those types of attacks in the selected tiles.

In [13]:
S1 = cn.spatioTemporal(
    log_store=STOREFILE,
    # Spatial
    coord1=coord1,
    coord2=coord2,
    grid=tiles,
    # Temporal
    year=year,
    month=month,
    day=day,
    init_date=init_date,
    end_date=end_date,
    freq=freq,
    # Event
    EVENT=event_dict['BEFIA']['col_name'],
    types=event_dict['BEFIA']['types'],
    threshold=event_dict['BEFIA']['threshold'])

S1.fit(csvPREF=event_dict['BEFIA']['csvname_prefix'])

100%|██████████| 47/47 [00:36<00:00,  1.30it/s]


#### c. Time Series for Armed Assault, Assassination, Hijacking, and Hostage Taking
`S2` fits for the attack types:
 - `Armed Assault`, 
 - `Hostage Taking (Barricade Incident)`, 
 - `Hijacking`, 
 - `Assassination`,
 - `Hostage Taking (Kidnapping) `.

Output is written to `AAHHH.csv`.

In [14]:
S2 = cn.spatioTemporal(
    log_store=STOREFILE,
    # Spatial
    coord1=coord1,
    coord2=coord2,
    grid=tiles,
    # Temporal
    year=year,
    month=month,
    day=day,
    init_date=init_date,
    end_date=end_date,
    freq=freq,
    # Event
    EVENT=event_dict['AAHHH']['col_name'],
    types=event_dict['AAHHH']['types'],
    threshold=event_dict['AAHHH']['threshold'])

S2.fit(csvPREF=event_dict['AAHHH']['csvname_prefix'])

100%|██████████| 47/47 [00:29<00:00,  1.58it/s]


### 3. Generate Triplet for Training and Split for Testing
Now we use the csv files created in previous steps (listed in `CSVfiles`) to generate the triplet files for training and split files for testing. 

 - The triplet files are generated with `readTS`.
    The training period is defined by `begin` and `end`. 
 - The split files are generated with `splitTS`. 
    The split files contains data from `begin` to `extended_end`. 
    The data for testing are those beyond the `end` and before the `extended_end`
    Here we set the `extended_end` to be one year beyond the `end`.

In [5]:
CSVfiles = [val['csvname_prefix'] + '.csv' for _, val in event_dict.items()]

begin, end, extended_end = init_date, '2015-12-31', end_date

# Make sure the triplet folder and split folder exist
os.mkdir(f'{path}/ntb/triplet/')
os.mkdir(f'{path}/ntb/split/')

In [6]:
# Triplet
triplet_fnames_prefix = f'{path}/ntb/triplet/TERROR_' + begin + '_' + end
cn.readTS(
    CSVfiles, 
    csvNAME=triplet_fnames_prefix, 
    BEG=begin, 
    END=end)

# Split
split_dirname = f'{path}/ntb/split/'
split_prefix = begin + '_' + extended_end + '_'
cn.splitTS(
    CSVfiles, 
    BEG=begin, 
    END=extended_end, 
    dirname=split_dirname, 
    prefix=split_prefix)

####  Optional cleanup of out-of-use files.

In [19]:
for CSVfile in CSVfiles:
    os.remove(CSVfile)
os.remove(STOREFILE)

## Model Generation
Now that we training and testing data ready, it is time to create the models.

**Input and Output of this step**
 - Input: training data (the triplet files) produced by `readTS`;
 - Output: model json files which each represnts a model.

### 1. Parameter setting

**Note:** It is highly recommended that we use absolute paths.

**Explanations:**
 - `PARTITION`: Since we work with event counts, a single partitioning at $-.5$ makes "no event" a $0$, and "any number of events more than $1$" a $1$.
 - `RUN_LOCAL`: 
     - If `False`, `xgModels` will produce a list of calls `program_calls.txt` that needs to be run to produce the models.
     - If `True`, `xgModels` will generate models locally. 
 - `NUM_RERUNS`: Since `XgenESeSS` is random, we usually run it several times to get the averaged result.
 - `XgenESeSS`: The location of the `XgenESeSS` binary.
     - it only work for Linux;

In [7]:
# File parameters
TS_PATH = triplet_fnames_prefix + '.csv' # The time series (data only)
NAME_PATH = triplet_fnames_prefix + '.coords' # The names for each time series
FILEPATH = f'{path}/ntb/models/' # Make sure to create a folder with name `FILEPATH` below
LOG_PATH = 'log.txt'

# XgenESSeS parameters
BEG = 1  # minimum delay considered
END = 60 # maximum delay considered
NUM_RERUNS = 2 # number of reruns
PARTITION = [.5] # partitioning points. 
XgenESeSS = f'{path}/cynet/bin/XgenESeSS'
RUN_LOCAL = True

# make sure a folder named `models` is created
os.mkdir(FILEPATH)

### 2. Running `xgModels` to generate model or model generating calls

In [21]:
XG = cn.xgModels(
    TS_PATH,
    NAME_PATH, 
    LOG_PATH,
    FILEPATH, 
    BEG, 
    END, 
    NUM_RERUNS, 
    PARTITION,
    XgenESeSS,
    RUN_LOCAL)
XG.run(workers=4)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


XgenESeSS Command 1 has startedXgenESeSS Command 2 has started

XgenESeSS Command 3 has startedXgenESeSS Command 4 has started

XgenESeSS Command 4 has finished
XgenESeSS Command 5 has started
XgenESeSS Command 3 has finished
XgenESeSS Command 6 has started
XgenESeSS Command 1 has finished
XgenESeSS Command 7 has started
XgenESeSS Command 2 has finished
XgenESeSS Command 8 has started
XgenESeSS Command 5 has finished
XgenESeSS Command 9 has started
XgenESeSS Command 6 has finished
XgenESeSS Command 10 has started
XgenESeSS Command 9 has finished
XgenESeSS Command 11 has started
XgenESeSS Command 8 has finished
XgenESeSS Command 12 has started
XgenESeSS Command 7 has finished
XgenESeSS Command 13 has started
XgenESeSS Command 10 has finished
XgenESeSS Command 14 has started
XgenESeSS Command 11 has finished
XgenESeSS Command 15 has started
XgenESeSS Command 12 has finished
XgenESeSS Command 16 has started
XgenESeSS Command 13 has finished
XgenESeSS Command 17 has started
XgenESeSS Comma

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 147.1min


XgenESeSS Command 43 has finished
XgenESeSS Command 47 has started
XgenESeSS Command 44 has finished
XgenESeSS Command 48 has started
XgenESeSS Command 46 has finished
XgenESeSS Command 49 has started
XgenESeSS Command 45 has finished
XgenESeSS Command 50 has started
XgenESeSS Command 48 has finished
XgenESeSS Command 51 has started
XgenESeSS Command 50 has finished
XgenESeSS Command 52 has started
XgenESeSS Command 49 has finished
XgenESeSS Command 53 has started
XgenESeSS Command 47 has finished
XgenESeSS Command 54 has started
XgenESeSS Command 51 has finished
XgenESeSS Command 55 has started
XgenESeSS Command 54 has finished
XgenESeSS Command 56 has started
XgenESeSS Command 52 has finished
XgenESeSS Command 57 has started
XgenESeSS Command 55 has finished
XgenESeSS Command 58 has started
XgenESeSS Command 56 has finished
XgenESeSS Command 59 has started
XgenESeSS Command 53 has finished
XgenESeSS Command 60 has started
XgenESeSS Command 57 has finished
XgenESeSS Command 61 has sta

[Parallel(n_jobs=4)]: Done 126 out of 126 | elapsed: 408.3min finished


## Model Evaluation
Here we evaluate our models by the AUC of the their prediction. 

The inner working of `run_pipeline`:
1. It first select `model_nums` number of models either by gamma or distance. 
    Then it creates a model_sel json file which is a filtered version of the models.
1. It applies the `cynet` binary to the model_sel files, which generates a log file containing predictions.
1. It applies the `flexroc` binary to the log files, once for each target type.
1. Finally, it writes test statistics (AUC, fpr, and, tpr) and output a `res_all.csv` file.

### 1. Parameter setting

**Explanation:**
 - `RUNLEN`: number of time steps in training and testing;
 - `FLEX_TAIL_LEN`: number of time steps in testing;
 - `model_nums`: maximum number of models to use in prediction;
 - `horizon`: prediction horizon;
 - `VARNAME`: the predicting variable types;
    Here we use individual variable types and `ALL` meaning all types of predicting variables are used together.
 - `gamma`: If `gamma` is true, the models are sorted with gamma (coefficient of causal dependence) and the best `model_nums` models will be used in the prediction;
 - To sort models by distance, use `distance=True` instead of `gamma=True` in `run_pipeline`;
 - `cores`: Number of cores running in parallel. 

In [8]:
# File parameters
MODEL_GLOB = f'{path}/ntb/models/*model.json'
RESPATH = f'{path}/ntb/models/*model*res'
DATA_PATH = os.path.join(split_dirname, split_prefix) # the split files path prefix 

# Prediction parameters
RUNLEN = len(pd.date_range(start=begin, end=extended_end, freq=freq))


# Now we get the start of the test period.
# Since the temporal resolution is 1 day, we just need to find the tomorrow of the training end.
from datetime import datetime, timedelta
start_of_test = datetime.strptime(end, '%Y-%m-%d') + timedelta(days=1)
start_of_test = start_of_test.date()

FLEX_TAIL_LEN = len(pd.date_range(start=start_of_test, end=extended_end, freq=freq))
model_nums = [20]
horizon = 7
VARNAME = list(set([fname.split('#')[-1] for fname in glob(DATA_PATH + "*")])) + ['ALL']

# Running parameters
# Make sure you have multi-core access when using cores greater than 1. 
cores = 4

print(f'run length (train + test) = {RUNLEN}\ntest length = {FLEX_TAIL_LEN}')

run length (train + test) = 1827
test length = 366


### 2. Run prediction pipeline

In [9]:
cn.run_pipeline(
    MODEL_GLOB,
    model_nums, 
    horizon, 
    DATA_PATH, 
    RUNLEN, 
    VARNAME, 
    RESPATH, 
    FLEX_TAIL_LEN=FLEX_TAIL_LEN,
    cores=cores,
    gamma=True)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.5min
[Parallel(n_jobs=4)]: Done 126 out of 126 | elapsed:  8.0min finished


Let use see a summary of the aucs

In [10]:
res = pd.read_csv('res_all.csv')
res[ (res['varsrc'] == 'ALL') & (res['auc'] < .999)]['auc'].describe()

count    126.000000
mean       0.778237
std        0.065529
min        0.567040
25%        0.751991
50%        0.780187
75%        0.807404
max        0.987983
Name: auc, dtype: float64

### Use recurrent neural network as a benchmark
Before we update `Cynet` to run with Python3, we have to run RNN in a separate Python3 notebook. 
The notebook is called `LSTM.ipynb`.
The LSTM result are saved to `LSTM_results` folder. 
The RNN AUCs is saved to file `LSTM_results/auc_rnn.csv`.
In the following cell, we compare the `Cynet` AUCs and the RNN AUCs.

In [15]:
# Load Cynet AUCs and process
df = pd.read_csv('res_all.csv')
df['model_id'] = df['loc_id'].apply(lambda x: int(x.split('/')[-1].split('m')[0]))
df = df[['model_id', 'lattgt1', 'lattgt2', 'lontgt1', 'lontgt2', 'varsrc', 'vartgt', 'auc']].copy()
df = pd.pivot_table(
    df, 
    values='auc', 
    index=['model_id', 'lattgt1', 'lattgt2', 'lontgt1', 'lontgt2', 'vartgt'],
    columns=['varsrc']).reset_index().set_index('model_id', drop=True)
srcs = [
    'ALL', 
    'Armed_Assault-Assassination-Hijacking-Hostage_Taking_Barricade_Incident-Hostage_Taking_Kidnapping',
    'Bombing_Explosion-Facility_Infrastructure_Attack',
    'VAR']
df['best_auc'] = df[srcs].max(axis=1)
df['best_src'] = df[srcs].idxmax(axis=1)
df['lat'] = (df['lattgt1'] + df['lattgt2']) / 2.0
df['lon'] = (df['lontgt1'] + df['lontgt2']) / 2.0

# Load RNN AUCs and combine
df_rnn = pd.read_csv('LSTM_results/auc_rnn.csv').set_index('model_id')
df_combined = df.join(df_rnn)
df_combined.to_csv('auc_cynet_rnn.csv', float_format='%.6f')

# prune unvalid data (remove sequence with no event in the out-of-sample period)

df_combined = df_combined[ (df_combined.best_auc < 0.9999) & (df_combined.rnn > 0)]
summary = df_combined[['best_auc', 'rnn']].describe().loc[['mean', '50%']]
summary

,best_auc,rnn
mean,0.834687,0.729708
50%,0.832638,0.733555


## Run Machine Learning Algorithms 
The cynet use a linear combination approach to aggregate the predictions from each source and form the final prediction for a target, but we can also apply machine learning algorithms in search for a better way to do the aggregation. 
For running machine learning algorithms, we need to run `cynet_chunker` to prepare csv files for running the machine leanring algorithms. 

### Parameter explanation
Comparing to running cynet prediction, we only possible change we have to make is the `model_nums`.
We can use a bigger number since machine learning algorithm can sometimes ignore noisy features
The parameter `cores` is currently a dummy paramters. 
The `cynet_chunker` uses $1$ core not matter what values you enter. 

Make sure to create a folder named `csvs` before running `cynet_chunker`.

In [11]:
os.mkdir('csvs')
model_nums_chunker = [200]
cn.cynet_chunker(
    MODEL_GLOB,
    model_nums_chunker, 
    horizon, 
    DATA_PATH, 
    RUNLEN, 
    VARNAME, 
    RESPATH,
    FLEX_TAIL_LEN=FLEX_TAIL_LEN,
    cores=1, # dummy parameter
    gamma=True,
    PARTITION=PARTITION[0]
)

  0%|          | 0/126 [00:00<?, ?it/s]

except
b'0\n'
except
b'0\n'
except
b'0\n'


  1%|          | 1/126 [00:41<1:25:42, 41.14s/it]

except
b'0\n'
except
b'0\n'
except
b'0\n'
except
b'0\n'


  2%|▏         | 2/126 [01:29<1:34:24, 45.68s/it]

...

except
b'0\n'
except
b'0\n'
except
b'0\n'
except
b'0\n'


 99%|█████████▉| 125/126 [1:45:05<00:46, 46.14s/it]

except
b'0\n'
except
b'0\n'
except
b'0\n'
except
b'0\n'


100%|██████████| 126/126 [1:46:16<00:00, 50.60s/it]

except
b'0\n'


## Machine leanring algorithm 
**Note:** Before Cynet is moved to Python3, we will have to run ML in a separate file.
I will copy the code here, but please don't run.

Currently, the code is in `MachineLearning.ipynb`.

In [14]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import csv
from glob import glob
import subprocess
import string
import random
import os
from functools import reduce

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import fbeta_score, make_scorer, r2_score
from sklearn.utils import shuffle

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier, RandomForestRegressor, RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

# rom lightgbm import LGBMRegressor

FLEXROC = f'{path}/cynet/bin/flexroc'

# init_date, end_date, freq = '2012-01-01', '2016-12-31', 'D'
# begin, end, extended_end = init_date, '2015-12-31', end_date
# RUNLEN = len(pd.date_range(start=begin, end=extended_end, freq=freq))
# FLEX_TAIL_LEN = len(pd.date_range(start=end, end=extended_end, freq=freq))
# print(f'run length = {RUNLEN};\nflex tail length = {FLEX_TAIL_LEN}.')
TRAINLEN = RUNLEN - FLEX_TAIL_LEN

def randomString(stringLength=8):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))


def run_one_combined(filenames, partition, regr_name):

    dfs = []
    for i, filename in enumerate(filenames):
        # Read in features csvs.
        df = pd.read_csv(filename, header=None)

        # Drop empty columns
        df = df.dropna()

        # Renaming columns
        cols = df.columns[1: -1]
        cols = ['timestep'] + ['F_{}_{}'.format(i, x) for x in cols] + ['target']
        df.columns = cols

        # Drop out-of-sample rows.
        df = df[df.target > -1]
        
        df['target'] = df['target'].apply(lambda x: 1 if x > partition else 0)

        dfs.append(df)

    df = reduce(lambda a, b: a.merge(b, how='inner', on=['timestep', 'target']), dfs)
    df.set_index('timestep', inplace=True)
    df.sort_index(inplace=True)


    # remove identical columns and 
    # move target to the last column 
    cols_to_keep = []
    for col in df:
        a = df[col].unique()
        if (col is not 'target') and (len(a) > 1):
            cols_to_keep.append(col)
    df = df[cols_to_keep + ['target']]


    # Train test split
    df_train = df[df.index < TRAINLEN]
    df_test = df[df.index >= TRAINLEN]

    data_train = df_train.values
    data_test = df_test.values

    X_train = data_train[:, :-1]
    y_train = data_train[:, -1]
    X_test = data_test[:, :-1]
    y_test = data_test[:, -1]


    if np.count_nonzero(y_test) == 0:
        return -1

    # Define Regressor or Classifier.
    regrs = {
        'ETree': ExtraTreesClassifier(
            n_estimators=5000, 
            max_depth=2, 
            min_samples_split=2, 
            class_weight='balanced'),
        'AdaB': AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=1000),
        'NN': MLPClassifier(alpha=1, max_iter=1000)
    }
    regr = regrs[regr_name]
    
    # Train the regressor.
    try: 
        regr.fit(X_train, y_train)

        y_prob = regr.predict_proba(X_test)
        p_thresholds = [p[1] for p in y_prob]

        # Very important, the y_test needs to be an int for flexroc to work properly.
        to_write = [[int(y_test[n]), p_thresholds[n]] for n in range(len(p_thresholds))]

        # Score or AUC
        prefix = filename.rstrip('.csv')
        log_filename = prefix + '.log'
        roc_filename = prefix + '.roc'
        with open(log_filename, 'w') as fh:
            writer = csv.writer(fh, delimiter=' ')
            for row in to_write:
                writer.writerow(row)
        auc_command = f'{FLEXROC} -i {log_filename}  -w 1 -x 0 -C 1 -L 1 -E 0 -f .2 -t .9 -r {roc_filename}'
        output = subprocess.check_output(auc_command, shell=True)

        # ------------add this line if working with python3 -----------------
        output = output.decode('utf8')
        # -------------------------------------------------------------------

        auc = float(output.split(' ')[1])
        return auc
    except:
        return -2


csv_folder = f'{path}/ntb/csvs'
partition = .5
regr_name = 'ETree'
output_file = f'{path}/ntb/ML_{regr_name}.csv'

filenames = glob(f'{csv_folder}/*model.csv')
model_ids, aucs = [], []
for i, filename in enumerate(filenames):
    model_id = filename.split('/')[-1][:-4].split('m')[0]
    auc = run_one_combined([filename], partition, regr_name)
    
    model_ids.append(model_id)
    aucs.append(auc)
    
    print(f'{i}/{len(filenames)}: model id = {model_id}, auc = {auc}')

df = pd.DataFrame(index=model_ids, data={'ml': aucs})
df.index.name = 'model_id'
df.sort_index(inplace=True)
df.to_csv(output_file)

df = df[df.ml > 0]
print(df.describe().loc()[['mean', '50%']])

            ml
mean  0.771644
50%   0.774193


## Simulation

### Get simulation file
In the next step, we will use the simulation file to get spatial relaxation and snapshot.

In [15]:
log_path = FILEPATH

cn.flexroc_only_parallel(
    '{}/*.log'.format(log_path),
    tpr_threshold=0.85,
    fpr_threshold=None,
    FLEX_TAIL_LEN=FLEX_TAIL_LEN, 
    cores=1)

mapper = cn.mapped_events('{}/*{}models#*#*.csv'.format(log_path, model_nums[0]))
mapper.concat_dataframes('{}/sim.csv'.format(log_path))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1008 out of 1008 | elapsed:  7.2min finished


Concating 504 files.


### Get spatial relaxation and plot snapshots

The following code isn't really runable right now. But after we move cynet to Python3, it will be okay.

In [27]:
import sys
sys.path.append(f'{path}/cynet_utils/')
import spatial as sp

import matplotlib.pyplot as plt
import matplotlib.cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

import geopandas as gpd
import contextily as ctx
from tqdm import tqdm

In [28]:
datetime_range = pd.date_range(start='2016-01-01', end='2016-12-31', freq='D')
print(f'total number of days = {len(datetime_range)}')
df = pd.read_csv(f'{path}/ntb/models/sim.csv')

lat_min, lat_max = df.lat1.min(), df.lat2.max()
lon_min, lon_max = df.lon1.min(), df.lon2.max()
print(f'lat = ({lat_min:.3f}, {lat_max:.3f})')
print(f'lon = ({lon_min:.3f}, {lon_max:.3f})')

day_min = 1462
df = df[ (df['day'] >= day_min) & (df['target'] != 'VAR') ]


print(df.target.unique())
events = [
    'Armed_Assault-Assassination-Hijacking-Hostage_Taking_Barricade_Incident-Hostage_Taking_Kidnapping',
    'Bombing_Explosion-Facility_Infrastructure_Attack'
]

day = 1517
time = datetime_range[day - day_min].date()
print(time)


Z = 0.06
parameters = {e: {} for e in events}
for var in events:
    types = [var]
    fn, tp, fp, ppv, sens, lon_, lat_, int_plot, int_, dfG, dfFN, dfTP, dfFP, dfTP0 = sp.get_prediction(
        df,
        day,
        types,
        lat_min,
        lat_max,
        lon_min,
        lon_max,
        radius=8,
        sigma=3.5,
        detail=1.2,
        miles=15,
        Z=Z)

    print('day={}, {}: fp={}, tp={}, fn={}; ppv={:.3f}, sens={:.3f}'
          .format(day, var, fp, tp, fn, ppv, sens))
    
    parameters[var].update({
        'fp': fp, 'tp': tp, 'fn': fn,
        'ppv': ppv, 'sens': sens,
        'lon': lon_, 'lat': lat_,
        'int': int_, 'df': dfG,
        'dffp': dfFP, 'dftp': dfTP, 'dffn': dfFN,
        'dftp_0': dfTP0
    })

total number of days = 366
lat = (-4.000, 49.000)
lon = (-6.000, 84.000)
['Armed_Assault-Assassination-Hijacking-Hostage_Taking_Barricade_Incident-Hostage_Taking_Kidnapping'
 'Bombing_Explosion-Facility_Infrastructure_Attack']
2016-02-25
tmporal comp: -->  tp  22  fp  50  fn  12
day=1517, Armed_Assault-Assassination-Hijacking-Hostage_Taking_Barricade_Incident-Hostage_Taking_Kidnapping: fp=1, tp=22, fn=4; ppv=0.957, sens=0.846
tmporal comp: -->  tp  27  fp  61  fn  12
day=1517, Bombing_Explosion-Facility_Infrastructure_Attack: fp=0, tp=27, fn=5; ppv=1.000, sens=0.844


In [29]:
cmap0 = sp.truncate_colormap(plt.cm.get_cmap('Reds'), 0.3, .9)
cmap1 = sp.truncate_colormap(plt.cm.get_cmap('Blues'), 0.3, .9)
bcolor0 = 'brown'
bcolor1 = 'blue'


parameters[events[0]]['cmap'] = cmap0
parameters[events[0]]['bcolor'] = bcolor0
parameters[events[0]]['mcolor'] = bcolor0
parameters[events[0]]['mtype'] = 'v'
parameters[events[1]]['cmap'] = cmap1
parameters[events[1]]['bcolor'] = bcolor1
parameters[events[1]]['mcolor'] = bcolor1
parameters[events[1]]['mtype'] = '^'

xlim = [-.1e7, 1.1e7]
ylim =[-.12e7, .92e7]

barTitleSize, barTickerSize = 20, 18
barWidth, barHeight = 1.4, 1.76
markerSize = 60
lighten_ratio = .2

width_height_ratio = (xlim[1] - xlim[0]) / (ylim[1] - ylim[0])
height = 10
width = width_height_ratio * height
fig = plt.figure(figsize=(width, height))
# A MUST-HAVE OR OTHERWISE THE DIMENSION OF THE PLOT ARE NOT UNIFORM
ax = fig.gca()
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)

var_title_map = {
    events[0]: 'Personnel',
    events[1]: 'Property'
}
for var in events:

    fp, tp, fn = parameters[var]['fp'], parameters[var]['tp'], parameters[var]['fn']
    lon_, lat_ = parameters[var]['lon'], parameters[var]['lat']
    int_, dfG = parameters[var]['int'], parameters[var]['dftp']

    dfG['Latitude'] = (dfG.lat1 + dfG.lat2) / 2
    dfG['Longitude'] = (dfG.lon1 + dfG.lon2) / 2
    dfG_ = dfG[['Latitude','Longitude']]
    GND = gpd.GeoDataFrame(
        dfG_, 
        geometry=gpd.points_from_xy(dfG_.Longitude, dfG_.Latitude))
    GND.crs= {'init' :'epsg:4326'}
    GND = GND.to_crs(epsg=3857)

    Longitude = []
    Latitude = []
    density = []
    for row in tqdm(np.arange(0,len(int_))):
        for col in np.arange(0,len(int_[row])):
            if int_[row, col] > 0:
                Longitude = np.append(Longitude,lon_[row,col])
                Latitude = np.append(Latitude,lat_[row,col])
                density = np.append(density,int_[row,col])

    df_density=pd.DataFrame(data={
        'Longitude': Longitude, 
        'Latitude': Latitude, 
        'density':density})

    wdf = gpd.GeoDataFrame(
        df_density, 
        geometry=gpd.points_from_xy(df_density.Longitude, df_density.Latitude))
    wdf.crs= {'init' :'epsg:4326'}
    wdf = wdf.to_crs(epsg=3857)

    ax = fig.gca()
    ax = wdf.plot(
        ax=ax,
        column='density',
        edgecolor='w',
        linewidth=0,
        cmap=parameters[var]['cmap'],
        alpha=.6, 
        zorder=5, 
        marker='s',
        markersize=135)

    ax=GND.plot(
        ax=ax, 
        marker=parameters[var]['mtype'],
        lw=1,
        # edgecolor='b',
        facecolors=parameters[var]['mcolor'],
        edgecolor=parameters[var]['mcolor'],
        # color=lighten_color(parameters[var]['mcolor'], lighten_ratio), 
        markersize=150, 
        alpha=.8,
        zorder=10)

    ctx.add_basemap(ax, source=ctx.sources.ST_TONER_BACKGROUND,alpha=.5)  ###IXC


# ======================== Time stamp ===================== START
ax.text(
    0.925, 0.05,
    str(time).split()[0], 
    transform=ax.transAxes,
    fontweight='bold',
    fontsize=20,
    color='w',
    verticalalignment='bottom', 
    horizontalalignment='right', 
    bbox=dict(
        boxstyle='round', 
        facecolor='midnightblue', 
        alpha=0.9)
)
# ======================== Time stamp ===================== END


# ================== Add bars of FN, TP, FP ==============START
width_ratio = barWidth / width
height_ratio = barHeight / height
parameters[events[0]]['bar_location'] = [0.625, 0.75, width_ratio, height_ratio]
parameters[events[1]]['bar_location'] = [0.85, 0.75, width_ratio, height_ratio]

plt.gcf().patches.extend([
    plt.Rectangle(
        (0.6, 0.7), 
        0.6, 0.25, 
        fill=True, 
        color='w', 
        alpha=0.9, 
        zorder=1,
        transform=fig.transFigure, 
        figure=plt.gcf())
])

for var in events:
    cmap = parameters[var]['cmap']
    bar_location = parameters[var]['bar_location']
    
    bar_data = np.array([
        parameters[var]['fn'], 
        parameters[var]['tp'], 
        parameters[var]['fp']
    ])
    bar_data = bar_data / bar_data.sum()

    
    ax2 = plt.gcf().add_axes(bar_location, zorder=20)
    ax2.patch.set_alpha(1)
    ax2.set_facecolor("white")
    
    ax2.bar(
        ['FN','TP','FP'],
        bar_data,
        color=parameters[var]['bcolor'],
        lw=0,
        zorder=20,
        alpha=.9)

    ax2.spines['bottom'].set_color('black')
    ax2.spines['top'].set_color('black') 
    ax2.spines['right'].set_visible(False) 
    ax2.spines['left'].set_visible(False) 
    ax2.tick_params(axis='x', colors='black', pad=8)
    ax2.tick_params(axis='y', colors='black')

    ax2.set_title(
        var_title_map[var], 
        color='black', 
        fontdict={'fontsize': barTitleSize, 'fontweight': 'bold'})
    ttl = ax2.title
    ttl.set_position([.5, 1.05])
    ax2.grid(True)

    for label in ax2.get_xticklabels():
        label.set_color('black')
        label.set_fontsize(barTickerSize)
        label.set_fontweight('bold')

    for label in ax2.get_yticklabels():
        label.set_color('black')
        label.set_fontsize(barTickerSize)
        label.set_fontweight('bold')

    ax2.tick_params(axis=u'both', which=u'both',length=0)
# ================== Add bars of FN, TP, FP ==============END

ax.axis('off')
plt.show()

# sp.saveFIG(f'snapshot_{day}.pdf')

100%|██████████| 58/58 [00:00<00:00, 7388.37it/s]


(-1000000.0, 11000000.0, -1200000.0, 9200000.0)